In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageChops, ImageEnhance
import numpy as np
from keras.models import load_model

# Load your pre-trained model
model = load_model('C:/Users/USER/Desktop/MODEL 28/Resnet 28.keras')
class_names = ['Fake', 'Real']

class ImageClassifierApp:
    def __init__(self, root):
        self.root = root
        self.img_reference = None
        self.prepared_img = None

        self.setup_ui()

    def prepare_image(self, image_path):
        image = Image.open(image_path).convert('RGB')
        image = image.resize((128, 128))  
        image.save('temp.jpg')  

        temp_image = Image.open('temp.jpg').convert('RGB')  
        ela_image = ImageChops.difference(image, temp_image)  

        extrema = ela_image.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1
        scale = 255.0 / max_diff

        ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

        return ela_image

    def classify_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            img = Image.open(file_path)
            img = img.resize((128, 128))  
            self.img_reference = ImageTk.PhotoImage(img)  

            image_label.config(image=self.img_reference)
            image_label.image = self.img_reference  

            self.prepared_img = self.prepare_image(file_path)
            predict_button.config(command=self.predict)  
    result_label = None
    def predict(self):
        if self.prepared_img is not None:
            prepared_img = np.array(self.prepared_img) / 255.0  
            prepared_img = np.expand_dims(prepared_img, axis=0)  
            prediction = model.predict(prepared_img)
            predicted_class = np.argmax(prediction, axis=1)[0]
            class_label = class_names[predicted_class]
            self.predicted_class_value.config(text=f"{class_label}")
            self.confidence_level_value.config(text=f"{np.amax(prediction) * 100:0.2f}%")

    def setup_ui(self):
        self.root.title("Fake/Real Image System Prototype")
        self.root.configure(bg='light gray')

        browse_button = tk.Button(self.root, text="Choose Image", command=self.classify_image, bg='dodger blue', fg='white', font=('Arial', 14))
        browse_button.pack(pady=20)

        global image_label, result_label, predict_button
        image_label = tk.Label(self.root, bg='light gray')
        image_label.pack(pady=20)

        result_frame = tk.Frame(self.root, bg='light gray')
        result_frame.pack(pady=20)

        predicted_class_label = tk.Label(result_frame, text="Predicted Class: ", bg='light gray', font=('Arial', 12))
        predicted_class_label.pack(side=tk.LEFT)

        self.predicted_class_value = tk.Label(result_frame, text="", bg='light gray', font=('Arial', 12))
        self.predicted_class_value.pack(side=tk.LEFT)

        confidence_level_label = tk.Label(result_frame, text="Confidence Level: ", bg='light gray', font=('Arial', 12))
        confidence_level_label.pack(side=tk.LEFT)

        self.confidence_level_value = tk.Label(result_frame, text="", bg='light gray', font=('Arial', 12))
        self.confidence_level_value.pack(side=tk.LEFT)

        predict_button = tk.Button(self.root, text="Predict", command=self.predict, bg='dodger blue', fg='white', font=('Arial', 14))
        predict_button.pack(pady=20)
        self.result_label = tk.Label(self.root, text="", bg='light gray', font=('Arial', 12))
        self.result_label.pack(pady=20)
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageClassifierApp(root)
    root.mainloop()
